### Step1. Downloading the tiwitter data from object storage and displaying first 5 rows

In [1]:
import urllib.request
import ast
import pandas as pd 

target_url="https://swift-yeg.cloud.cybera.ca:8080/v1/AUTH_233e84cd313945c992b4b585f7b9125d/geeky-summit/tweets.txt"
file_name="tweets.txt"
urllib.request.urlretrieve(target_url, file_name)
tweets = []
with open(file_name, 'r') as f:
    for lines in f.readlines():
        tweets.append(ast.literal_eval(lines))
frame  = pd.DataFrame(tweets)
frame.head()

,coordinates,created_at,extended_tweet,hashtags,lang,name,screen_name,text,user_location,user_mentions
0,None,Fri Nov 02 21:01:56 +0000 2018,@Symin16 I’d like to know who,[],en,♠,jessmayumba85,@Symin16 I’d like to know who,Toronto ✈ Calgary,"[{'indices': [0, 8], 'id': 1031043960968364034..."
1,None,Fri Nov 02 21:02:01 +0000 2018,@TwoCanSamAdams Legit. There are still chunks ...,[],en,hannahrae cuddleslug,thimblewad,@TwoCanSamAdams Legit. There are still chunks ...,YYC,"[{'indices': [0, 15], 'id': 406677625, 'screen..."
2,"{'type': 'Point', 'coordinates': [-114.007914,...",Fri Nov 02 21:02:05 +0000 2018,Can you recommend anyone for this #job in #Cal...,"[{'indices': [34, 38], 'text': 'job'}, {'indic...",en,TMJ - CAL Manuf Jobs,tmj_cal_manuf,Can you recommend anyone for this #job in #Cal...,Calgary,[]
3,None,Fri Nov 02 21:02:10 +0000 2018,Kids See Ghosts: https://t.co/J4phIy2CQy love ...,[],en,Sunny Rai,TheSunsRay,Kids See Ghosts: https://t.co/J4phIy2CQy love ...,🌎📱,[]
4,"{'type': 'Point', 'coordinates': [-114.0740046...",Fri Nov 02 21:02:13 +0000 2018,"See our latest #Calgary, AB #job and click to ...","[{'indices': [15, 23], 'text': 'Calgary'}, {'i...",en,TMJ-CAL Retail Jobs,tmj_cal_retail,"See our latest #Calgary, AB #job and click to ...",Calgary,[]


### Step2. Understanding the input data

In [2]:
print("Number of rows/columns:",frame.shape, "\n")
for column in frame.columns:
    if column=="user_mentions":
        rownum=1
    else :
        rownum=2
    print("Column name: {colname} \n Column type: {coltype} \n Column data(example): {coldata} \n\n"
          .format(colname=column, coltype=type(frame[column].iloc[rownum]),coldata=frame[column].iloc[rownum]))

Number of rows/columns: (14793, 10) 

Column name: coordinates 
 Column type: <class 'dict'> 
 Column data(example): {'type': 'Point', 'coordinates': [-114.007914, 51.004583]} 


Column name: created_at 
 Column type: <class 'str'> 
 Column data(example): Fri Nov 02 21:02:05 +0000 2018 


Column name: extended_tweet 
 Column type: <class 'str'> 
 Column data(example): Can you recommend anyone for this #job in #Calgary, AB? https://t.co/20JiTB3cRi #SupplyChain #Hiring #CareerArc 


Column name: hashtags 
 Column type: <class 'list'> 
 Column data(example): [{'indices': [34, 38], 'text': 'job'}, {'indices': [42, 50], 'text': 'Calgary'}, {'indices': [80, 92], 'text': 'SupplyChain'}, {'indices': [93, 100], 'text': 'Hiring'}, {'indices': [101, 111], 'text': 'CareerArc'}] 


Column name: lang 
 Column type: <class 'str'> 
 Column data(example): en 


Column name: name 
 Column type: <class 'str'> 
 Column data(example): TMJ - CAL Manuf Jobs 


Column name: screen_name 
 Column type: <class '

 - **coordinates** - user coordinates, can be None or Python type Dict(needs to be split to two columns - longitude and latitude)
 - **created_at**  - creation date, Python type String (needs to be converted to type Date + timezone needs to be changed from UTC to MST)
 - **extended_tweet** - tweet text, Python type String(`extended_tweet` and `text` are the similar columns, `extended_tweet` has more data, we will use it instead of `text`)
 - **hashtags** - hashtags, Python type List of Dicts(needs to be converted to String)
 - **lang** - language, Python type String
 - **name** - twitter account name, Python type String
 - **screen_name** - screen name, Python type String
 - **text** - tweet text, Python type String(`extended_tweet` and `text` are the similar columns, `extended_tweet` has more data, we will use it instead of `text`)
 - **user_location** - location, Python type String
 - **user_mentions** - user mentions, Python type List of Dicts(needs to be converted to String)

### Step4. Wrangling the data
#### Convert column "created_at"  from String to Date and change timezone from UTC to MST

In [3]:
print(frame["created_at"].iloc[2])
frame["created_at_date"]= pd.to_datetime(frame["created_at"])
frame["created_at_date"]=frame["created_at_date"].dt.tz_localize('UTC').dt.tz_convert('America/Edmonton')
print(frame["created_at_date"].iloc[2])

Fri Nov 02 21:02:05 +0000 2018
2018-11-02 15:02:05-06:00


#### Convert column "coordinates" from Dict tow columns "latitude" and "longitude"

In [4]:
print(frame["coordinates"].head())
def convertCoordToList(v):
    if v is None:
        v=[]
    else:
        v= v['coordinates']
    return v
frame['coordinates_list']= frame['coordinates'].map(convertCoordToList)
frame[['latitude','longitude']] = pd.DataFrame(frame['coordinates_list'].values.tolist())
print(frame[['longitude','latitude']].head())

0                                                 None
1                                                 None
2    {'type': 'Point', 'coordinates': [-114.007914,...
3                                                 None
4    {'type': 'Point', 'coordinates': [-114.0740046...
Name: coordinates, dtype: object
   longitude    latitude
0        NaN         NaN
1        NaN         NaN
2  51.004583 -114.007914
3        NaN         NaN
4  50.997882 -114.074005


#### Convert column "hashtags"  from Dict to Str

In [5]:
print(frame["hashtags"].iloc[2])
def convertHashToString(v):
    if v == []:
        v=' '
    else:
        v=' '.join([d['text'] for d in v])
    return v
frame['hashtags_string']= frame['hashtags'].map(convertHashToString)
print("\n",frame["hashtags_string"].iloc[2])

[{'indices': [34, 38], 'text': 'job'}, {'indices': [42, 50], 'text': 'Calgary'}, {'indices': [80, 92], 'text': 'SupplyChain'}, {'indices': [93, 100], 'text': 'Hiring'}, {'indices': [101, 111], 'text': 'CareerArc'}]

 job Calgary SupplyChain Hiring CareerArc


#### Excersise: convert column "user_mentions" from Dict to Str

In [6]:
print(frame["user_mentions"].iloc[1])
def convertUserToString(v):
    if v == []:
        v=' '
    else:
        v=' '.join([d['screen_name'] for d in v])## type something here (this line will be hidden)
    return v
frame['user_string']= frame['user_mentions'].map(convertUserToString)
print("\n",frame["user_string"].iloc[1])

[{'indices': [0, 15], 'id': 406677625, 'screen_name': 'TwoCanSamAdams', 'id_str': '406677625', 'name': 'TwoCan Jorge'}]

 TwoCanSamAdams


### Step5. Cleaning the data

In [7]:
pd.set_option('max_colwidth', 700)
frame['extended_tweet'].head(10)

0                                                                                                                                                                                                                                                                                @Symin16 I’d like to know who
1                                                                                                                                  @TwoCanSamAdams Legit. There are still chunks in the sink and I can’t stand to be in the room, even though it doesn’t smell like puke anymore. I’m just so done with today.
2                                                                                                                                                                                              Can you recommend anyone for this #job in #Calgary, AB? https://t.co/20JiTB3cRi #SupplyChain #Hiring #CareerArc
3                                                                                          

####  Remove URLS 

In [8]:
frame['extended_tweet']=frame['extended_tweet'].str.replace('http\S+|www.\S+', '', case=False)

#### Excersise: remove '/n' (new line symbol) from 'extended_tweet' column

In [9]:
frame['extended_tweet']=frame['extended_tweet'].str.replace('\n',' ') ### this will be hidden

#### Remove user mentions and hashtags from the 'extended_tweet' column and save it as 'extended_tweet_cleaned' column

In [10]:
frame['extended_tweet_cleaned']=frame['extended_tweet'].str.replace('[@#]\w+', ' ',case=False)
frame['extended_tweet_cleaned'].head(10)

0                                                                                                                                                                                                                                I’d like to know who
1                                                                                         Legit. There are still chunks in the sink and I can’t stand to be in the room, even though it doesn’t smell like puke anymore. I’m just so done with today.
2                                                                                                                                                                                                Can you recommend anyone for this   in  , AB?       
3                                                                                                                                                                                                                   Kids See Ghosts:  love this track
4               

#### Addictional cleaning (emoji,punctuation, numbers, special characters) will be applied  in the later notebooks

### Step 6. Subset only columns we need and save the cleaned dataset

In [11]:
frame_cleaned=frame[['created_at_date','hashtags_string','user_string','user_location',
                     'lang','longitude','latitude','name', 'screen_name','extended_tweet','extended_tweet_cleaned']]
frame_cleaned.head()

,created_at_date,hashtags_string,user_string,user_location,lang,longitude,latitude,name,screen_name,extended_tweet,extended_tweet_cleaned
0,2018-11-02 15:01:56-06:00,,Symin16,Toronto ✈ Calgary,en,NaN,NaN,♠,jessmayumba85,@Symin16 I’d like to know who,I’d like to know who
1,2018-11-02 15:02:01-06:00,,TwoCanSamAdams,YYC,en,NaN,NaN,hannahrae cuddleslug,thimblewad,"@TwoCanSamAdams Legit. There are still chunks in the sink and I can’t stand to be in the room, even though it doesn’t smell like puke anymore. I’m just so done with today.","Legit. There are still chunks in the sink and I can’t stand to be in the room, even though it doesn’t smell like puke anymore. I’m just so done with today."
2,2018-11-02 15:02:05-06:00,job Calgary SupplyChain Hiring CareerArc,,Calgary,en,51.004583,-114.007914,TMJ - CAL Manuf Jobs,tmj_cal_manuf,"Can you recommend anyone for this #job in #Calgary, AB? #SupplyChain #Hiring #CareerArc","Can you recommend anyone for this in , AB?"
3,2018-11-02 15:02:10-06:00,,,🌎📱,en,NaN,NaN,Sunny Rai,TheSunsRay,Kids See Ghosts: love this track,Kids See Ghosts: love this track
4,2018-11-02 15:02:13-06:00,Calgary job,,Calgary,en,50.997882,-114.074005,TMJ-CAL Retail Jobs,tmj_cal_retail,"See our latest #Calgary, AB #job and click to apply: barista - Store# 54359, CHINOOK CENTRE FOOD COURT - #Retail #Hiring #CareerArc","See our latest , AB and click to apply: barista - Store# 54359, CHINOOK CENTRE FOOD COURT -"


In [12]:
frame_cleaned.to_csv('tweets_cleaned.csv',index=False)